# Training pipeline for TFRS  2tower model 

```
tensorflow==2.10.1
tensorflow-cloud==0.1.16
tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-hub==0.12.0
tensorflow-io==0.27.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.8.0
tensorflow-probability==0.18.0
tensorflow-recommenders==0.7.2
tensorflow-serving-api==2.8.3
tensorflow-transform==1.8.0
```

In [ ]:
# !pip install kfp
# !pip install google-cloud-pipeline-components 

In [ ]:
# ! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
# ! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"
# ! python3 -c "import google.cloud.aiplatform; print('aiplatform SDK version: {}'.format(google.cloud.aiplatform.__version__))"

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")

VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1


In [2]:
import os
import json
from datetime import datetime
from time import time
import pandas as pd
# disable INFO and DEBUG logging everywhere
import logging
import time
from pprint import pprint

logging.disable(logging.WARNING)

from google.cloud import aiplatform as vertex_ai
from google.cloud import storage

# Pipelines
from typing import Any, Callable, Dict, NamedTuple, Optional, List
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google_cloud_pipeline_components.types import artifact_types

# Kubeflow SDK
# TODO: fix these
from kfp.v2 import dsl
import kfp
import kfp.v2.dsl
from kfp.v2.google import client as pipelines_client
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component)

storage_client = storage.Client(project=PROJECT_ID)

vertex_ai.init(project=PROJECT_ID,location=LOCATION)

In [3]:
# PREFIX = 'spotify-2tower'
APP='sp'
MODEL_TYPE='2tower'
FRAMEWORK = 'tfrs'
MODEL_VERSION = 'jtv10'
PIPELINE_VERSION = 'v0'
MODEL_ROOT_NAME = f'{APP}-{MODEL_TYPE}-{FRAMEWORK}-{MODEL_VERSION}-{PIPELINE_VERSION}'

print(f"MODEL_ROOT_NAME: {MODEL_ROOT_NAME}")

MODEL_ROOT_NAME: sp-2tower-tfrs-jtv10-v0


## Write Train files

In [4]:
# # Make the training subfolder
# ! rm -rf {REPO_DOCKER_PATH_PREFIX}/trainer
# ! mkdir {REPO_DOCKER_PATH_PREFIX}/trainer
# ! touch {REPO_DOCKER_PATH_PREFIX}/trainer/__init__.py

REPO_DOCKER_PATH_PREFIX = 'src'

In [5]:
# Docker definitions for training
IMAGE_NAME = f'{MODEL_ROOT_NAME}-training'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

DOCKERNAME = 'tfrs'
REPO_DOCKER_PATH_PREFIX = 'src'
MACHINE_TYPE ='e2-highcpu-32'
FILE_LOCATION = './src'

print(f"IMAGE_URI: {IMAGE_URI}")

IMAGE_URI: gcr.io/hybrid-vertex/sp-2tower-tfrs-jtv10-v0-training


In [6]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/cloudbuild.yaml

steps:
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', '$_IMAGE_URI', '$_FILE_LOCATION', '-f', '$_FILE_LOCATION/Dockerfile.$_DOCKERNAME']
images:
- '$_IMAGE_URI'

Overwriting src/cloudbuild.yaml


In [7]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/two_tower_jt/train_config.py

PROJECT_ID = 'hybrid-vertex'
NEW_ADAPTS = 'True'
USE_CROSS_LAYER = False
USE_DROPOUT = 'False'
SEED = 1234
MAX_PLAYLIST_LENGTH = 5         # this should improve performance vs 375
EMBEDDING_DIM = 128   
PROJECTION_DIM = 50  
SEED = 1234
DROPOUT_RATE = 0.33
MAX_TOKENS = 20000

Overwriting src/two_tower_jt/train_config.py


In [8]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/two_tower_jt/requirements.txt
google-cloud-aiplatform==1.18.0
tensorflow-recommenders==0.7.2
tensorboard==2.10.1
tensorboard-data-server==0.6.1
tensorboard-plugin-profile==2.8.0
scann
google-cloud-aiplatform[cloud_profiler]

Overwriting src/two_tower_jt/requirements.txt


In [9]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/Dockerfile.{DOCKERNAME}

FROM tensorflow/tensorflow:2.10.1-gpu

WORKDIR /src

# Copies the trainer code to the docker image.
COPY two_tower_jt/* two_tower_jt/ 

RUN pip install -r two_tower_jt/requirements.txt

# # Sets up the entry point to invoke the trainer.
# # ENTRYPOINT ["python", "-m", "two_tower_jt.task"]

Overwriting src/Dockerfile.tfrs


## Build Custom Train Image

In [10]:
print(f"DOCKERNAME: {DOCKERNAME}")
print(f"IMAGE_URI: {IMAGE_URI}")
print(f"FILE_LOCATION: {FILE_LOCATION}")
print(f"MACHINE_TYPE: {MACHINE_TYPE}")

DOCKERNAME: tfrs
IMAGE_URI: gcr.io/hybrid-vertex/sp-2tower-tfrs-jtv10-v0-training
FILE_LOCATION: ./src
MACHINE_TYPE: e2-highcpu-32


In [11]:
!pwd

/home/jupyter/jw-repo/spotify_mpd_two_tower


In [13]:
!tree /home/jupyter/jw-repo/spotify_mpd_two_tower/src

/home/jupyter/jw-repo/spotify_mpd_two_tower/src
├── Dockerfile.tfrs
├── cloudbuild.yaml
├── two_tower_jt
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-37.pyc
│   │   └── two_tower.cpython-37.pyc
│   ├── interactive_train.py
│   ├── requirements.txt
│   ├── task.py
│   ├── train_config.py
│   └── two_tower.py
└── vocab_pipes
    ├── __pycache__
    │   ├── adapt_fixed_text_layer_vocab.cpython-37.pyc
    │   ├── adapt_ragged_text_layer_vocab.cpython-37.pyc
    │   ├── config.cpython-37.pyc
    │   └── create_master_vocab.cpython-37.pyc
    ├── adapt_fixed_text_layer_vocab.py
    ├── adapt_ragged_text_layer_vocab.py
    ├── config.py
    └── create_master_vocab.py

4 directories, 18 files


In [14]:
! gcloud builds submit --config src/cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME,_IMAGE_URI=$IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

Creating temporary tarball archive of 1171 file(s) totalling 47.6 GiB before compression.
^C


Command killed by keyboard interrupt



# Pipeline Components

In [ ]:
os.getcwd()

In [ ]:
REPO_DOCKER_PATH_PREFIX = 'src'
PIPELINES_SUB_DIR = 'train_pipes'

In [ ]:
! rm -rf {REPO_DOCKER_PATH_PREFIX}/{PIPELINES_SUB_DIR}
! mkdir {REPO_DOCKER_PATH_PREFIX}/{PIPELINES_SUB_DIR}

## Build Custom Image

In [ ]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{PIPELINES_SUB_DIR}/build_custom_image.py

import kfp
from typing import Any, Callable, Dict, NamedTuple, Optional, List
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, Metrics)
@kfp.v2.dsl.component(
    base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest",
    packages_to_install=[
        "google-cloud-build"
    ],
)
def build_custom_image(
    project: str,
    artifact_gcs_path: str,
    docker_name: str,
    app_dir_name: str,
    custom_image_uri: str,
) -> NamedTuple('Outputs', [
    ('custom_image_uri', str),
]):
    # TODO: make output Artifact for image_uri
    """
    custom pipeline component to build custom image using
    Cloud Build, the training/serving application code, and dependencies
    defined in the Dockerfile
    """
    
    import logging
    import os

    from google.cloud.devtools import cloudbuild_v1 as cloudbuild
    from google.protobuf.duration_pb2 import Duration

    # initialize client for cloud build
    logging.getLogger().setLevel(logging.INFO)
    build_client = cloudbuild.services.cloud_build.CloudBuildClient()
    
    # parse step inputs to get path to Dockerfile and training application code
    _gcs_dockerfile_path = os.path.join(artifact_gcs_path, f"{docker_name}") # Dockerfile.XXXXX
    _gcs_script_dir_path = os.path.join(artifact_gcs_path, f"{app_dir_name}/") # "trainer/"
    
    logging.info(f"_gcs_dockerfile_path: {_gcs_dockerfile_path}")
    logging.info(f"_gcs_script_dir_path: {_gcs_script_dir_path}")
    
    # define build steps to pull the training code and Dockerfile
    # and build/push the custom training container image
    build = cloudbuild.Build()
    build.steps = [
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", "-r", _gcs_script_dir_path, "."],
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", _gcs_dockerfile_path, "Dockerfile"],
        },
        # enabling Kaniko cache in a Docker build that caches intermediate
        # layers and pushes image automatically to Container Registry
        # https://cloud.google.com/build/docs/kaniko-cache
        # {
        #     "name": "gcr.io/kaniko-project/executor:latest",
        #     # "name": "gcr.io/kaniko-project/executor:v1.8.0",        # TODO; downgraded to avoid error in build
        #     # "args": [f"--destination={training_image_uri}", "--cache=true"],
        #     "args": [f"--destination={training_image_uri}", "--cache=false"],
        # },
        {
            "name": "gcr.io/cloud-builders/docker",
            "args": ['build','-t', f'{custom_image_uri}', '.'],
        },
        {
            "name": "gcr.io/cloud-builders/docker",
            "args": ['push', f'{custom_image_uri}'], 
        },
    ]
    # override default timeout of 10min
    timeout = Duration()
    timeout.seconds = 7200
    build.timeout = timeout

    # create build
    operation = build_client.create_build(project_id=project, build=build)
    logging.info("IN PROGRESS:")
    logging.info(operation.metadata)

    # get build status
    result = operation.result()
    logging.info("RESULT:", result.status)

    # return step outputs
    return (
        custom_image_uri,
    )

## Custom train job

In [ ]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{PIPELINES_SUB_DIR}/train_custom_model.py

import kfp
from typing import Any, Callable, Dict, NamedTuple, Optional, List
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, Metrics)
@kfp.v2.dsl.component(
    base_image='python:3.9',
    packages_to_install=[
        'google-cloud-aiplatform==1.17.0',
        'tensorflow==2.9.2',
        'tensorflow-recommenders==0.7.0',
        'numpy',
        'google-cloud-storage',
    ],
    # output_component_file="./pipelines/train_custom_model.yaml",
)
def train_custom_model(
    project: str,
    model_version: str,
    pipeline_version: str,
    model_name: str, 
    worker_pool_specs: dict,
    # vocab_dict_uri: str, 
    train_output_gcs_bucket: str,                         # change to workdir?
    training_image_uri: str,
    tensorboard_resource_name: str,
    service_account: str,
    experiment_name: str,
    experiment_run: str,
) -> NamedTuple('Outputs', [
    ('query_tower_dir_uri', str),
    ('candidate_tower_dir_uri', str),
    # ('candidate_index_dir_uri', str),
]):
    
    from google.cloud import aiplatform as vertex_ai
    import logging
    import numpy as np
    
    vertex_ai.init(
        project=project,
        location='us-central1',
    )
    
    JOB_NAME = f'train-{model_name}'
    logging.info(f'JOB_NAME: {JOB_NAME}')
    
    BASE_OUTPUT_DIR = f'gs://{train_output_gcs_bucket}/{experiment_name}/{experiment_run}'
    logging.info(f'BASE_OUTPUT_DIR: {BASE_OUTPUT_DIR}')
    
    # logging.info(f'vocab_dict_uri: {vocab_dict_uri}')
    
    logging.info(f'tensorboard_resource_name: {tensorboard_resource_name}')
    logging.info(f'service_account: {service_account}')
    logging.info(f'worker_pool_specs: {worker_pool_specs}')
  
    job = vertex_ai.CustomJob(
        display_name=JOB_NAME,
        worker_pool_specs=worker_pool_specs,
        staging_bucket=BASE_OUTPUT_DIR,
    )
    
    logging.info(f'Submitting train job to Vertex AI...')

    job.run(
        tensorboard=tensorboard_resource_name,
        service_account=f'{service_account}',
        restart_job_on_worker_restart=False,
        enable_web_access=True,
        sync=False,
    )
    
    # TODO: this is hardcoded, but created in train job
    query_tower_dir_uri = f"gs://{train_output_gcs_bucket}/{experiment_name}/{experiment_run}/model-dir/query_tower" 
    candidate_tower_dir_uri = f"gs://{train_output_gcs_bucket}/{experiment_name}/{experiment_run}/model-dir/candidate_tower"
    # candidate_index_dir_uri = f"gs://{output_dir_gcs_bucket_name}/{experiment_name}/{experiment_run}/candidate-index"
    
    return (
        f'{query_tower_dir_uri}',
        f'{candidate_tower_dir_uri}',
        # f'{candidate_index_dir_uri}',
    )

## Prepare Job Specs

### Vertex Train: workerpool specs

In [ ]:
def prepare_worker_pool_specs(
    image_uri,
    args,
    cmd,
    replica_count=1,
    machine_type="n1-standard-16",
    accelerator_count=1,
    accelerator_type="ACCELERATOR_TYPE_UNSPECIFIED",
    reduction_server_count=0,
    reduction_server_machine_type="n1-highcpu-16",
    reduction_server_image_uri="us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest",
):

    if accelerator_count > 0:
        machine_spec = {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        }
    else:
        machine_spec = {"machine_type": machine_type}

    container_spec = {
        "image_uri": image_uri,
        "args": args,
        "command": cmd,
    }

    chief_spec = {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": container_spec,
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            "replica_count": replica_count - 1,
            "machine_spec": machine_spec,
            "container_spec": container_spec,
        }
        worker_pool_specs.append(workers_spec)
    if reduction_server_count > 1:
        workers_spec = {
            "replica_count": reduction_server_count,
            "machine_spec": {
                "machine_type": reduction_server_machine_type,
            },
            "container_spec": {"image_uri": reduction_server_image_uri},
        }
        worker_pool_specs.append(workers_spec)

    return worker_pool_specs

### Accelerators and Device Strategy

In [ ]:
# # Single machine, single GPU
WORKER_MACHINE_TYPE = 'a2-highgpu-1g'
REPLICA_COUNT = 1
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
PER_MACHINE_ACCELERATOR_COUNT = 1
REDUCTION_SERVER_COUNT = 0                                                      
REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"
DISTRIBUTE_STRATEGY = 'single'

# # # Single Machine; multiple GPU
# WORKER_MACHINE_TYPE = 'a2-highgpu-4g' # a2-ultragpu-4g
# REPLICA_COUNT = 1
# ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
# PER_MACHINE_ACCELERATOR_COUNT = 4
# REDUCTION_SERVER_COUNT = 0                                                      
# REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"
# DISTRIBUTE_STRATEGY = 'mirrored'

# # # Multiple Machines, 1 GPU per Machine
# WORKER_MACHINE_TYPE = 'n1-standard-16'
# REPLICA_COUNT = 9
# ACCELERATOR_TYPE = 'NVIDIA_TESLA_T4'
# PER_MACHINE_ACCELERATOR_COUNT = 1
# REDUCTION_SERVER_COUNT = 10                                                      
# REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"
# DISTRIBUTE_STRATEGY = 'multiworker'

### Training Config

In [ ]:
# ====================================================
# Pipeline output repo 
# ====================================================
OUTPUT_BUCKET = 'jt-tfrs-output-v2'
OUTPUT_GCS_URI =f'gs://{OUTPUT_BUCKET}'

EXPERIMENT_PREFIX = 'pipe-dev'                                                  # custom identifier for organizing experiments
EXPERIMENT_NAME=f'{EXPERIMENT_PREFIX}-{MODEL_TYPE}-{FRAMEWORK}-{MODEL_VERSION}'
RUN_NAME = f'run-{time.strftime("%Y%m%d-%H%M%S")}'

# Stores pipeline executions for each run
PIPELINE_ROOT_PATH = f'gs://{OUTPUT_BUCKET}/{MODEL_ROOT_NAME}/pipeline_root'
print('PIPELINE_ROOT_PATH: {}'.format(PIPELINE_ROOT_PATH))

TB_RESOURCE_NAME = 'projects/934903580331/locations/us-central1/tensorboards/7336372589079560192'

# =================================================
# Data sources
# =================================================
CANDIDATE_FILE_DIR = 'spotify-data-regimes'
CANDIDATE_PREFIX = 'candidates/' 

TRAIN_DIR = 'spotify-data-regimes' 
# TRAIN_DIR_PREFIX = 'train_v9/'
TRAIN_DIR_PREFIX = 'valid_v9/'

VALID_DIR = 'spotify-data-regimes' 
VALID_DIR_PREFIX = 'valid_v9/'

# =================================================
# train image
# =================================================
# Existing image URI or name for image to create
# IMAGE_URI = 'gcr.io/hybrid-vertex/sp-2tower-tfrs-v15-v0-training'
# IMAGE_NAME = f'{MODEL_NAME}-training'
# IMAGE_URI = f'gcr.io/{PROJECT}/{IMAGE_NAME}'

# =================================================
# train job config
# =================================================
SEED = 1234
VALID_FREQUENCY = 10
# VALID_SIZE = 20_000

MAX_TOKENS = 20000

NUM_EPOCHS = 1
BATCH_SIZE = 1024*16 # batch_size = 1024*16
LEARNING_RATE = 0.01

# MAX_PADDING = 375
EMBEDDING_DIM = 128
PROJECTION_DIM = 50

DROPOUT_RATE = 0.33
LAYER_SIZES = '[64,32]'

WORKER_CMD = ["python", "trainer/task.py"]
# WORKER_CMD ["python", "-m", "trainer.task"]

WORKER_ARGS = [
    f'--project={PROJECT_ID}',
    f'--train_output_gcs_bucket={OUTPUT_BUCKET}',
    f'--train_dir={TRAIN_DIR}',
    f'--train_dir_prefix={TRAIN_DIR_PREFIX}',
    f'--valid_dir={VALID_DIR}',
    f'--valid_dir_prefix={VALID_DIR_PREFIX}',
    f'--candidate_file_dir={CANDIDATE_FILE_DIR}',
    f'--candidate_files_prefix={CANDIDATE_PREFIX}',
    f'--experiment_name={EXPERIMENT_NAME}',
    f'--experiment_run={RUN_NAME}',
    f'--num_epochs={NUM_EPOCHS}',
    f'--batch_size={BATCH_SIZE}',
    f'--embedding_dim={EMBEDDING_DIM}',
    f'--projection_dim={PROJECTION_DIM}',
    f'--layer_sizes={LAYER_SIZES}',
    f'--learning_rate={LEARNING_RATE}',
    # f'--valid_frequency={VALID_FREQUENCY}',
    f'--distribute={DISTRIBUTE_STRATEGY}',
    f'--model_version={MODEL_VERSION}',
    f'--pipeline_version={PIPELINE_VERSION}',
    f'--seed={SEED}',
    f'--max_tokens={MAX_TOKENS}',
    f'--tb_resource_name={TB_RESOURCE_NAME}',
]

WORKER_POOL_SPECS = prepare_worker_pool_specs(
    image_uri=IMAGE_URI,
    args=WORKER_ARGS,
    cmd=WORKER_CMD,
    replica_count=REPLICA_COUNT,
    machine_type=WORKER_MACHINE_TYPE,
    accelerator_count=PER_MACHINE_ACCELERATOR_COUNT,
    accelerator_type=ACCELERATOR_TYPE,
    reduction_server_count=REDUCTION_SERVER_COUNT,
    reduction_server_machine_type=REDUCTION_SERVER_MACHINE_TYPE,
)

from pprint import pprint
pprint(WORKER_POOL_SPECS)

In [ ]:
!export PWD=pwd
!export PIPELINE_ROOT_PATH=PIPELINE_ROOT_PATH

! echo $PWD
! echo $PIPELINE_ROOT_PATH

In [ ]:
# # # copy training Dockerfile
# ! gsutil cp $PWD/src/Dockerfile.tfrs $BUCKET_URI/$VERSION/src
!gsutil cp $PWD/Dockerfile.tfrs $PIPELINE_ROOT_PATH
!gsutil cp $PWD/cloudbuild.yaml $PIPELINE_ROOT_PATH

# # # copy training application code
! gsutil -m cp -r $PWD/two_tower_jt/* $PIPELINE_ROOT_PATH/trainer

print(f"Copied training application code and Dockerfile to {PIPELINE_ROOT_PATH}")

## Build Pipeline

In [ ]:
PIPELINE_TAG = f'2tower-recsys-{PIPELINE_VERSION}'
print("PIPELINE_TAG:", PIPELINE_TAG)

PIPELINE_NAME = f'modeling-{MODEL_VERSION}-{PIPELINE_TAG}'.replace('_', '-')
print("PIPELINE_NAME:", PIPELINE_NAME)

In [ ]:
from src.pipelines import build_custom_image, train_custom_model

@kfp.v2.dsl.pipeline(
    name=f'{PIPELINE_NAME}'.replace('_', '-')
)
def pipeline(
    project: str,
    project_number: str,
    location: str,
    service_account: str,
    model_version: str,
    pipeline_version: str,
    train_image_uri: str,
    train_output_gcs_bucket: str,
    gcs_train_script_path: str,
    model_display_name: str,
    train_dockerfile_name: str,
    train_dir: str,
    train_dir_prefix: str,
    valid_dir: str,
    valid_dir_prefix: str,
    candidate_file_dir: str,
    candidate_files_prefix: str,
    experiment_name: str,
):
    
    from kfp.v2.components import importer_node
    from google_cloud_pipeline_components.types import artifact_types
    
    # ========================================================================
    # Build Custom Train Image
    # ========================================================================
    
    # build_custom_train_image_op = (
    #     build_custom_train_image.build_custom_train_image(
    #         project=project,
    #         gcs_train_script_path=gcs_train_script_path,
    #         training_image_uri=train_image_uri,
    #         train_dockerfile_name=train_dockerfile_name,
    #     )
    #     .set_display_name("Build custom train image")
    #     .set_caching_options(False)
    # )


    run_train_task_op = (
        train_custom_model.train_custom_model(
            project=project,
            model_version=model_version,
            pipeline_version=pipeline_version,
            model_name=model_display_name,
            worker_pool_specs=WORKER_POOL_SPECS, 
            train_output_gcs_bucket=train_output_gcs_bucket,
            # vocab_dict_uri=build_vocabs_string_lookups_op.outputs['vocab_gcs_uri'],
            experiment_name=experiment_name,
            experiment_run=experiment_run,
            training_image_uri=train_image_uri,                                                                       # build_custom_train_image_op.outputs['training_image_uri'],
            tensorboard_resource_name="projects/934903580331/locations/us-central1/tensorboards/3482390819678191616", # create_tensorboard_op.outputs['tensorboard_resource_name'],
            service_account=service_account,
        )
        .set_display_name("2Tower Training")
        .set_caching_options(True)
        # .after(build_custom_train_image_op)
    )
    
    # ========================================================================
    # Upload Query and Candidate Towers
    # ========================================================================
    
    import_unmanaged_query_model_task = (
        importer_node.importer(
            artifact_uri=run_train_task_op.outputs['query_tower_dir_uri'],
            artifact_class=artifact_types.UnmanagedContainerModel,
            metadata={
                'containerSpec': {
                    'imageUri': 'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-10:latest',
                },
            },
        )
        .set_display_name("Import Query Tower")
        .after(run_train_task_op)
        .set_caching_options(True)
    )
    
    query_model_upload_op = (
        gcc_aip.ModelUploadOp(
            project=project,
            location=location,
            display_name=f'query-tower-{model_display_name}',
            unmanaged_container_model=import_unmanaged_query_model_task.outputs["artifact"],
            labels={"tower": "query"},
        )
        .after(import_unmanaged_query_model_task)
        .set_display_name("Upload Query Tower")
        .set_caching_options(True)
    )
    
    import_unmanaged_candidate_model_task = (
        importer_node.importer(
            artifact_uri=run_train_task_op.outputs['candidate_tower_dir_uri'],
            artifact_class=artifact_types.UnmanagedContainerModel,
            metadata={
                'containerSpec': {
                    'imageUri': 'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-10:latest',
                },
            },
        )
        .set_display_name("Import Candidate Tower")
        .after(run_train_task_op)
        .set_caching_options(True)
    )

    candidate_model_upload_op = (
        gcc_aip.ModelUploadOp(
            project=project,
            location=location,
            display_name=f'candidate-tower-{model_display_name}',
            unmanaged_container_model=import_unmanaged_candidate_model_task.outputs["artifact"],
            labels={"tower": "candidate"},
        )
        # .after(import_unmanaged_query_model_task)
        .set_display_name("Upload Query Tower to Vertex")
        .set_caching_options(True)
    )

In [ ]:
! rm -f pipelines/custom_container_pipeline_spec.json

PIPELINE_JSON_SPEC_PATH = "./src/pipelines/custom_container_pipeline_spec.json"

kfp.v2.compiler.Compiler().compile(
    pipeline_func=pipeline, package_path=PIPELINE_JSON_SPEC_PATH,
)

## Submit Pipeline

In [ ]:
PROJECT_NUMBER='934903580331'
vpc_network_name = 'ucaip-haystack-vpc-network'
SERVICE_ACCOUNT = '934903580331-compute@developer.gserviceaccount.com'

# ========================================================================
# Submit Pipeline Job
# ========================================================================
# # "gs://spotify-tfrecords-blog/tfrecords_v1/train/output-00000-of-00796.tfrecord"
# # gs://spotify-tfrs-dir/small-dataset/output-00000-of-00796.tfrecord

overwrite = True
    
if not PIPELINES.get('train') or overwrite:
    response = pipeline_client.create_run_from_job_spec(
        job_spec_path=PIPELINE_JSON_SPEC_PATH,
        enable_caching=True,
        network=f'projects/{PROJECT_NUMBER}/global/networks/{vpc_network_name}',            #
        # service_account=SERVICE_ACCOUNT,                                                 
        parameter_values={
            'project': PROJECT_ID,
            'project_number': PROJECT_NUMBER,
            'location': LOCATION,
            'model_version': MODEL_VERSION,
            'pipeline_version': PIPELINE_VERSION,
            'model_display_name': MODEL_ROOT_NAME,
            'pipeline_tag': PIPELINE_TAG,
            'gcs_train_script_path': TRAIN_APP_CODE_PATH,
            'train_image_uri': IMAGE_URI,
            'train_output_gcs_bucket': OUTPUT_BUCKET,
            'train_dir': TRAIN_DIR,
            'train_dir_prefix': TRAIN_DIR_PREFIX,
            'valid_dir': VALID_DIR,
            'valid_dir_prefix': VALID_DIR_PREFIX,
            'candidate_file_dir': CANDIDATE_FILE_DIR,
            'candidate_files_prefix': CANDIDATE_PREFIX,
            # 'vpc_network_name': vpc_network_name,
            'train_dockerfile_name': DOCKERNAME_TRAIN,
            'experiment_name': EXPERIMENT_NAME,
            'experiment_run': RUN_NAME,
            'service_account': SERVICE_ACCOUNT,
        },
        pipeline_root=f'{PIPELINE_ROOT_PATH}/{PIPELINE_VERSION}',
    )
    PIPELINES['train'] = response['name']